In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from src.scripts.merge_cmu import clean_cmu
from src.scripts.category_analysis import *

In [6]:
# Load the dataset CMU
cmu_movies = pd.read_csv("all_movies.csv")

# Load the dataset merge_imdb
df = pd.read_csv("adapted_movies.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'all_movies.csv'

In [5]:
df.head()

,imdbID,isAdult,movie_name,movie_date,box_office,runtime,language,countries,genres,author_book,year_book,rating,numVotes
0,tt0000679,0,thefairylogueandradio-plays,1908,NaN,120.0,English Language,United States of America,"Silent film, Black-and-white",L. Frank Baum,1900,5.2,76
1,tt0002646,0,atlantis,1913,NaN,113.0,"English Language, Danish Language",Denmark,"Silent film, Drama, Indie, Black-and-white",Gerhart Hauptmann,1912,6.5,500
2,tt0003022,0,ivanhoe,1913,NaN,NaN,"Silent film, English Language",United States of America,"Swashbuckler films, Silent film, Drama, Adventure",Sir Walter Scott,1820,5.6,97
3,tt0004099,0,"hismajesty,thescarecrowofoz",1914,NaN,60.0,English Language,United States of America,"Silent film, Adventure, Children's/Family, Bla...",L. Frank Baum,1900,5.3,533
4,tt0004182,0,thejungle,1914,NaN,NaN,"Silent film, English Language",NaN,Silent film,Upton Sinclair,1906,6.8,70


## Category Analysis Evolution

In [8]:
_, df_ada = analysis_by_category(df)
_,df_cmu = analysis_by_category(cmu_movies)



KeyError: 'MovieGenre'

In [7]:
genre_categories = load_genre_categories('data/genre_categories.json')

In [ ]:
evolution_adapt = category_evolution(df_ada)
evolution_cmu = category_evolution(df_cmu)

In [ ]:
for category in genre_categories:
    fig = plt.figure(figsize=(10, 7))
    sns.histplot(evolution_adapt[category], stat='percent', kde=True, bins=100)
    plt.title(f"Distribution of {category} Movies Over Time", fontsize=16)
    plt.xlabel("Year", fontsize=14)
    plt.ylabel("Nb of movies released", fontsize=14)
    plt.grid(True) 
    

In [ ]:
for category in genre_categories:
    fig = plt.figure(figsize=(10, 7))
    sns.histplot(evolution_cmu[category], stat='percent', kde=True, bins=100)
    plt.title(f"Distribution of {category} Movies Over Time", fontsize=16)
    plt.xlabel("Year", fontsize=14)
    plt.ylabel("Nb of movies released", fontsize=14)
    plt.grid(True) 